In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
timeseries_data = pd.read_csv('/pfs/work9/workspace/scratch/ma_tofuchs-GraphWave-Seminar/Datasets/Mannheim/imputed_dataset_grin_full.csv', index_col=0)

In [3]:
# Compute split indices
num_samples = len(timeseries_data)
train_size = int(num_samples * 0.6)
val_size = int(num_samples * 0.2)

In [4]:
# Split the data
train_data = timeseries_data.iloc[:train_size]
val_data = timeseries_data.iloc[train_size: train_size + val_size]
test_data = timeseries_data.iloc[train_size + val_size:]

# Combine train_data and val_data
train_data = pd.concat([train_data, val_data], ignore_index=False)

In [5]:
# Compute the mean for each sensor in the training set
sensor_means = train_data.mean()

# Forecast the mean values for the test set (replicating the mean for each timestamp)
mean_forecast = pd.DataFrame(np.tile(sensor_means.values, (len(test_data), 1)), 
                             index=test_data.index, 
                             columns=test_data.columns)

In [8]:
# Compute per-sensor error metrics
mae_per_sensor = mean_absolute_error(test_data, mean_forecast, multioutput='raw_values')
rmse_per_sensor = np.sqrt(mean_squared_error(test_data, mean_forecast, multioutput='raw_values'))
mape_per_sensor = (np.abs((test_data - mean_forecast) / test_data)).replace([np.inf, -np.inf], np.nan).mean() * 100

# Store per-sensor metrics in a dataframe
sensor_error_metrics = pd.DataFrame({
    "MAE": mae_per_sensor,
    "RMSE": rmse_per_sensor,
    "MAPE (%)": mape_per_sensor
}, index=test_data.columns)

In [9]:
# Compute overall metrics
overall_mae = mae_per_sensor.mean()
overall_rmse = rmse_per_sensor.mean()
overall_mape = mape_per_sensor.mean()

# Store overall metrics in a separate dataframe
overall_error_metrics = pd.DataFrame({
    "MAE": [overall_mae],
    "RMSE": [overall_rmse],
    "MAPE (%)": [overall_mape]
})

In [10]:
sensor_error_metrics

,MAE,RMSE,MAPE (%)
21649702,568.975508,671.399905,376.614623
21649767,1017.089097,1146.339207,137.905000
21673581,205.476193,236.100780,221.604721
21732938,274.092491,403.805008,267.496581
24555545,767.968727,878.516792,162.193699
25117313,223.570522,261.466455,173.303508
25117359,714.764221,841.312824,344.797856
27088492,795.755008,912.910420,214.909028
27428132,390.731517,448.622763,158.010965
27428349,125.591822,147.284200,130.210463


In [11]:
overall_error_metrics

,MAE,RMSE,MAPE (%)
0,548.429964,631.613887,271.267066
